#  Dynamic Data Collection for GANS Case Study via API 

TASK_1: Load weather data from OpenWeatherMap via API and load it into the SQL database for the GANS case study.
To ccomplish: Extract geo and weather data from Open Weather API that matches each city

TASK_2: Load flight from Aero Data box API and load it into the SQL database for the GANS case study.
To acomplish: Extract airports and from each airport arrivals that match with cities

In [201]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import json
from datetime import datetime, timedelta
import pymysql
import os

In [203]:
def dynamic_db_setup():
    
    """
    This function sets up the dynamic databases by performing the following steps:
    1. Establishes a database connection using the connection_setup() function.
    2. Retrieves city names from the SQL database.
    3. Retrieves latitude and longitude data from the SQL database.
    4. Retrieves airport data from the SQL database.
    5. Calls an API to retrieve weather data using the geo_data_df.
    6. Calls an API to retrieve flights per hour data using the airports_df.
    7. Sends the weather data to SQL using the send_weather() function.
    8. Sends the flights per hour data to SQL using the send_flights() function.
    9. Returns a success message.

    Returns:
    - str: A message indicating the successful update of data.
    """
    
    connection_string = connection_setup()
        
    # get city names from SQL
    cities = pd.read_sql("cities",con=connection_string)
    city_lst = cities.city_name.to_list()
    # Read latitudes and longitudes from SQL 
    geo_data_df = pd.read_sql("geo_data", con=connection_string)
    latitudes = geo_data_df.latitude.to_list()
    longitudes =geo_data_df.longitude.to_list()
    # Get airports from sql
    airports_df = pd.read_sql("airports", con=connection_string)
    # dynamic data
    weather_df = weather_api_call(geo_data_df)
    flights_per_hour_df = arrivals_api_call(airports_df)
    #send to sql
    send_weather(weather_df, connection_string)
    send_flights(flights_per_hour_df, connection_string)
    
    return "data sucessfully updated"

## Scrape Data

### Weather

In [208]:
#Use of weather API and use only single values for each city. 3h timestamp with temperature and sky description

def weather_api_call(geo_data_df):
    """
    Scrape weather data for a list of cities using the OpenWeatherMap API.
    Args:
        geo_data_df (pandas.DataFrame): A DataFrame containing geo data with columns 'city_name', 'latitude', and 'longitude'.
    Returns:
        pandas.DataFrame: A DataFrame containing the scraped weather data with columns 'city_name', 'forecast_time', 'temperature', 'sky', 'rain_in_last_3h', and 'wind_speed'.
    """
    API_key = "91d16ed59eecaf4ad6eb1e9d19482549"
    weather = {"city_id": [], "forecast_time": [], "temperature": [], "sky": [], "rain_in_last_3h": [], "wind_speed": []}
    today = datetime.now().date()
    tomorrow = str((today + timedelta(days=1)))

    for i in range(geo_data_df["latitude"].count()):
        lat = geo_data_df["latitude"].to_list()[i]
        lon = geo_data_df["longitude"].to_list()[i]
        
        url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric"
        response = requests.get(url)
        json_data = response.json()
        
        for item in json_data["list"]:
            weather["city_id"].append(geo_data_df["city_id"].to_list()[i])
            weather["forecast_time"].append(item.get("dt_txt", None))
            weather["temperature"].append(item["main"].get("temp", None))
            weather["sky"].append(item["weather"][0].get("description", None))
            weather["rain_in_last_3h"].append(item.get("rain", {}).get("3h", 0))
            weather["wind_speed"].append(item["wind"].get("speed", None))
    
    weather_df = pd.DataFrame(weather)
   
    # transform the timestamps to datetime
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])

    # create a new column with only the day
    weather_df["forecast_days"] = weather_df.forecast_time.dt.strftime("%Y-%m-%d")
    weather_df["forecast_hour"] = weather_df.forecast_time.dt.strftime("%H")
    weather_df["forecast_days"] = pd.to_datetime(weather_df["forecast_days"])
    weather_df["forecast_hour"] = weather_df["forecast_hour"].astype(int)
    weather_df = weather_df.loc[weather_df["forecast_days"]== tomorrow,:]
    
    #weather_df = weather_df.loc[]
    return weather_df

### Flights

In [210]:
def arrivals_api_call(airports_df):
    iata = airports_df["iata"].to_list()
    city_id = airports_df["city_id"].to_list()
    today  = datetime.now().date()
    tomorrow = (today + timedelta(days=1))
    half_day = [["00:00","11:59"],["12:00","23:59"]]
    headers = {
        "X-RapidAPI-Key": "4b54815ac4mshe11ca88893efc1ep170ae5jsn49ac4b2d04a5",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
    querystring = {
        "direction": "Arrival",
        "withCancelled": "false",
        "withCodeshared": "true",
        "withCargo": "false",
        "withPrivate": "false",
        "withLocation": "false"
    }
    flights = {"city_id": [], "date": [], "arrival": []}

#  For 12 hours 0 - 12 
    for hours in half_day:
        for i in range(len(iata)):
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata[i]}/{tomorrow}T{hours[0]}/{tomorrow}T{hours[1]}"
            response = requests.get(url, headers=headers, params=querystring)
            json_data = response.json()  
            # Parse the JSON response
            if "arrivals" in json_data:
                for item in json_data["arrivals"]:
                    flights["city_id"].append(city_id[i])
                    flights["date"].append(tomorrow)
                    flights["arrival"].append(item["movement"]['scheduledTime'].get("local", None))

            else:
                print(f"no arrivals on{dates[j]}")
    new_times = [re.sub(r"\+01:00","",flight) for flight in flights["arrival"]]
    flights["arrival"] = new_times    
    flights_df = pd.DataFrame(flights)
    
    flights_df["arrival"] = pd.to_datetime(flights_df["arrival"])
    flights_df["date"] = pd.to_datetime(flights_df["date"])
    flights_df["hour"] = flights_df["arrival"].dt.strftime("%H")
    flights_df["hour"] = pd.to_numeric(flights_df["hour"])
    
    flights_per_hour_df = flights_df.groupby(["city_id","date", "hour"]).agg(number_of_flights =("arrival","count")).reset_index()
    return flights_per_hour_df

### Load to the SQL database 

In [ ]:
def connection_setup():
    """
    Sets up the connection string for connecting to a MySQL database.
    Returns:
    - connection_string (str): The connection string for connecting to the MySQL database.
    """
    schema = "gans_locations"
    host = "host"
    user = "root"
    password =os.environ.get("Password")
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

In [214]:
def send_weather(weather, connection_string):
    """
    Writes an update of weahter data and day from a pandas DataFrame to a MySQL database table. Replaces - existing Values
    for weather updates, and append new days to the Database
    
    Args:
    - weather: set of weather data
    - unique days from the weather table
    -connection_string: String to connect with MYSQL

    Returns:
    - None
    """


    weather.to_sql('weather_data',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [215]:
def send_flights(flights_per_hour_df, connection_string):
    """
    Writes flight data per day from a pandas DataFrame to a MySQL database table. appends the values for each new day
   
    Args:
    - flights_per hour: dataframe grouped by city, day and hour
    -connection_string: String to connect with MYSQL

    Returns:
    - None
    """


    flights_per_hour_df.to_sql('flight_data',
                  if_exists='append',
                  con=connection_string,
                  index=False)

## Final

In [217]:
dynamic_db_setup()

'data sucessfully updated'

In [218]:
pd.read_sql("flight_data",con=connection_string)

,city_id,date,hour,number_of_flights
0,1,2024-01-18,6,2
1,1,2024-01-18,7,9
2,1,2024-01-18,8,8
3,1,2024-01-18,9,11
4,1,2024-01-18,10,16
...,...,...,...,...
173,5,2024-01-18,18,62
174,5,2024-01-18,19,112
175,5,2024-01-18,20,83
176,5,2024-01-18,21,10


In [219]:
pd.read_sql("airports", con=connection_string)

,number_airports,iata,airport_name,city_id
0,1,BER,Berlin Brandenburg,1
1,1,HAM,Hamburg,2
2,1,MUC,Munich,3
3,1,CGN,Cologne Bonn,4
4,1,FRA,Frankfurt-am-Main,5
